In [ ]:
import os
import json
import sys
import pandas as pd
import torch

PROJECT_ROOT = "/media/ttoxopeus/basic_UNet"

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"✅ Added project root to sys.path:\n   {PROJECT_ROOT}")
else:
    print(f"ℹ️ Project root already in sys.path:\n   {PROJECT_ROOT}")


from src.training.eval import evaluate
from src.models.unet import UNet
from src.pruning.model_inspect import model_to_dataframe_with_l1, get_pruning_masks_blockwise, compute_actual_prune_ratios, compute_l1_norms, compute_l1_stats, inspect_model_l1
from src.pruning.rebuild import rebuild_pruned_unet, find_prev_conv_name

✅ Added project root to sys.path:
   /media/ttoxopeus/basic_UNet


/media/ttoxopeus/miniconda3/envs/nnunet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the model
model = UNet(in_ch=1, out_ch=4, enc_features=[64, 128, 256, 512, 1024, 2048], dec_features=None, bottleneck_out=None)
state = torch.load("/media/ttoxopeus/basic_UNet/results/UNet_ACDC/exp21_new_train_summary/baseline/training/final_model.pth", map_location="cpu")
model.load_state_dict(state)
model.eval()
print("✅ Model loaded successfully.")

✅ Model loaded successfully.


In [3]:
df_summary = inspect_model_l1(model, save_dir="./l1_analysis")
pd.set_option("display.max_rows", None)
display(df_summary)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(64, 1, 3, 3)",1,64,576,1.499611,0.946125,2.081812,0.264741,None,None
1,encoders.0.net.3,Conv2d,"(64, 64, 3, 3)",64,64,36864,12.144828,11.527526,12.737896,0.283900,None,None
2,encoders.1.net.0,Conv2d,"(128, 64, 3, 3)",64,128,73728,12.342955,11.591101,13.121187,0.308784,None,None
3,encoders.1.net.3,Conv2d,"(128, 128, 3, 3)",128,128,147456,17.707615,16.855709,18.442064,0.320407,None,None
4,encoders.2.net.0,Conv2d,"(256, 128, 3, 3)",128,256,294912,17.778671,16.942108,18.501942,0.308145,None,None
5,encoders.2.net.3,Conv2d,"(256, 256, 3, 3)",256,256,589824,26.066133,24.837185,27.364107,0.441375,None,None
6,encoders.3.net.0,Conv2d,"(512, 256, 3, 3)",256,512,1179648,26.081352,24.829105,27.246586,0.490790,None,None
7,encoders.3.net.3,Conv2d,"(512, 512, 3, 3)",512,512,2359296,39.360184,36.468506,42.498108,1.055625,None,None
8,encoders.4.net.0,Conv2d,"(1024, 512, 3, 3)",512,1024,4718592,39.478466,35.839115,44.255554,1.098097,None,None
9,encoders.4.net.3,Conv2d,"(1024, 1024, 3, 3)",1024,1024,9437184,61.519203,53.979794,69.441895,2.613565,None,None


In [3]:
norms = compute_l1_norms(model)
stats = compute_l1_stats(norms)
df = model_to_dataframe_with_l1(model, stats, remove_nan_layers=False)

pd.set_option("display.max_rows", None)
display(df)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(64, 1, 3, 3)",1,64,576,1.499611,0.946125,2.081812,0.264741,None,None
1,encoders.0.net.3,Conv2d,"(64, 64, 3, 3)",64,64,36864,12.144828,11.527526,12.737896,0.283900,None,None
2,encoders.1.net.0,Conv2d,"(128, 64, 3, 3)",64,128,73728,12.342955,11.591101,13.121187,0.308784,None,None
3,encoders.1.net.3,Conv2d,"(128, 128, 3, 3)",128,128,147456,17.707615,16.855709,18.442064,0.320407,None,None
4,encoders.2.net.0,Conv2d,"(256, 128, 3, 3)",128,256,294912,17.778671,16.942108,18.501942,0.308145,None,None
5,encoders.2.net.3,Conv2d,"(256, 256, 3, 3)",256,256,589824,26.066133,24.837185,27.364107,0.441375,None,None
6,encoders.3.net.0,Conv2d,"(512, 256, 3, 3)",256,512,1179648,26.081352,24.829105,27.246586,0.490790,None,None
7,encoders.3.net.3,Conv2d,"(512, 512, 3, 3)",512,512,2359296,39.360184,36.468506,42.498108,1.055625,None,None
8,encoders.4.net.0,Conv2d,"(1024, 512, 3, 3)",512,1024,4718592,39.478466,35.839115,44.255554,1.098097,None,None
9,encoders.4.net.3,Conv2d,"(1024, 1024, 3, 3)",1024,1024,9437184,61.519203,53.979794,69.441895,2.613565,None,None
